# Single-Qubit Systems
<hr></hr>

In the introductory chapter to [quantum bits and circuits](https://learnquantum.io/chapters/02_quantum_computing/02_01_bits_to_qubits.html), we presented the model of a qubit inspired by the probabilistic behavior of the spin of an electron. However, we only described how the electron behaves when its spin was pointing in the $\texttt{+}z, \texttt{-}z, \texttt{+}x, \texttt{-}x$ directions (described by states $|0\rangle, |1\rangle, |+\rangle, |-\rangle$, respectively). 

But what if the spin is pointing along some arbitrary angle with respect to the $\texttt{+}z$ direction? We did mention that the probability of measuring spin-up or spin-down depends on this angle, but did not give a quantitative rule for how electrons behave. 

In this chapter, we will not only expand on this idea, but will generalize the state of the spin of an electron pointing in any direction, which represents the most general way to express the state of a qubit. We will then discuss the different operations we can apply to a qubit to transform it from one arbitrary state to another, and lastly talk about the Bloch Sphere, which is a graphical representation of the mathematical description of the qubit's state.

## 1. Generalizing the Qubit


### 1.1 Real Probability Amplitudes

If we, once again, look at the Stern-Gerlach experiment, the results from performing repeated experiments on an electron with its spin at an angle $\theta$ with respect to the $\texttt{+}z$ axis show that, the probability of the electron deflecting in the $\texttt{+}z$ direction (i.e., measuring state $|0\rangle$) is given by the cosine squared of the angle $\theta/2$:

$$ \mathbb{P}_{0} = \cos^2\left(\frac{\theta}{2}\right) $$

Similarly, the probability of the electron deflecting in the $\texttt{-}z$ direction (i.e., measuring state $|1\rangle$) is equal the sine squared of the angle:

$$ \mathbb{P}_{1} = \sin^2\left(\frac{\theta}{2}\right) $$

<img src="images/02_03_01_stern-gerlach_angle_prob.png" align = "center" width="360"/>

From a probability standpoint, these expressions work since the sum of sine and cosine squared for any common angle always add to one, just like probabilities ought to do:

$$
\begin{aligned}
\mathbb{P}_{0} &+ \mathbb{P}_{1} = 1
\\
\\
\cos^2\left(\frac{\theta}{2}\right) &+ \sin^2\left(\frac{\theta}{2}\right) = 1
\end{aligned}
$$

Therefore, as discussed in the section on [probability amplitudes](https://learnquantum.io/chapters/02_quantum_computing/02_01_bits_to_qubits.html#from-probabilities-to-probability-amplitudes), we can construct a statevector that represents the state of a qubit (electron spin) by making sure that when the selected probability amplitudes are squared, we obtain the empirical probabilities that match those from observed results via experimentation (as specified by the [Born rule](https://en.wikipedia.org/wiki/Born_rule)):

$$ 
\begin{aligned}
|q\rangle &= \begin{bmatrix} \cos\left(\frac{\theta}{2}\right) \\ \sin\left(\frac{\theta}{2}\right) \end{bmatrix}
\\
\\
|q\rangle &= \cos\left(\frac{\theta}{2}\right)|0\rangle + \sin\left(\frac{\theta}{2}\right) |1\rangle 
\end{aligned}
$$

And this, of course, checks out for the four cases we have explored so far:

| Spin Direction | State | Angle $\theta$ [rad] | $\cos\left(\frac{\theta}{2}\right)$ |  $\sin\left(\frac{\theta}{2}\right)$ |
| :-: | :-: | :-: | :-: | :-: |
| $\texttt{+}z$ | $\vert 0\rangle$ | $0$ | $1$ | $0$ |
| $\texttt{-}z$ | $\vert 1\rangle$ | $\pi$ |  $0$ | $1$ |
| $\texttt{+}x$ | $\vert +\rangle$ | $\frac{\pi}{2}$ | $\frac{1}{\sqrt{2}}$ | $\frac{1}{\sqrt{2}}$ |
| $\texttt{-}x$ | $\vert -\rangle$ | $\frac{3\pi}{2}$ | $\frac{1}{\sqrt{2}}$ | $-\frac{1}{\sqrt{2}}$ |

But more generally, it works for any angle. Let's look at the specific example where $\theta = \pi/3$. The expectation is that we will get the following respective probabilities for measuring states $|0\rangle$ and $|1\rangle$:

$$ 
\begin{aligned}
&\mathbb{P}_{0} = \cos^2\left(\frac{\pi/3}{2}\right) = \left(\frac{\sqrt{3}}{2}\right)^2 = \frac{3}{4} = 0.75
\\
\\
&\mathbb{P}_{1} = \sin^2\left(\frac{\pi/3}{2}\right) = \left(\frac{1}{2}\right)^2 = \frac{1}{4} = 0.25
\end{aligned}
$$


And now let's use the `Statevector` class in qiskit to construct this statevector, and see if the measurement probabilities match our expectation:

In [1]:
import numpy as np
from qiskit.quantum_info import Statevector

θ = np.pi/3               # Spin angle wrt to +z axis
α = np.cos(θ/2)           # Probability amplitude associated with |0⟩
β = np.sin(θ/2)           # Probability amplitude associated with |1⟩

q = Statevector([α, β])   # Construct statevector |q⟩ = α|0⟩ + β|1⟩ = [α β]ᵀ

probs = q.probabilities() # Extract expected probs array [P₀, P₁]
print(probs)

[0.75 0.25]


The result above is the calculation of the ideal probabilities, but we can also sample the statevector (as if we were performing measurements in this system) and get a distribution of results that should be approximately give us $75\%$ of measuring $0$ and $25\%$ of measuring $1$:

In [5]:
counts = q.sample_counts(shots=1000) # "Measure" statevector 1000 times and 
                                     # return dict {'0': num_of_0s, '1': num_of_0s}
print(counts)

{'0': 730, '1': 270}


This generalization of the qubit, where the probability amplitudes are given by the $\cos$ and $\sin$ squared of the angle $\theta/2$, works well to describe the results of the Stern-Gerlach experiment; however, there is a bit of an issue. In this particular case, we are only concerned about the direction of the spin on the $xz$-plane because we are shooting the electrons along the $y$ direction. But how do we describe the spin of the electron if it is aligned with, let's say, the $\texttt{+}y$ axis? Well, experiments show that, if we were to measure the spin of such an electron with a field oriented along the $z$ axis, we will still see the electron deflecting up or down with $50\%$ probability (just like we do for spin in the $\texttt{+}x$ direction). As a matter of fact, this is true no matter which orientation (given by the angle $\varphi$) the electron has in the $xy$-plane:

<img src="images/02_03_02_elec_phi_angle.png" align = "center" width="520"/>

So, how do we represent such states? After all, for spin along the $\pm x$ directions, we defined the probability amplitudes by taking the positive and negative square roots of the probability of measuring spin-up and spin-down, so what do we do for spin along, let's say the $\pm y$ directions?

<div style="text-align: center;"> <strong> ¡Complex Numbers to the Rescue! </strong> </div>

### 1.2 Complex Probability Amplitudes

Let us start by recalling the reason why we chose states $|+\rangle$ and $|-\rangle$ to be of the form:

$$ 
|+\rangle = \frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}|1\rangle \quad \text{ and } \quad
|-\rangle = \frac{1}{\sqrt{2}}|0\rangle - \frac{1}{\sqrt{2}}|1\rangle.
$$

First off, we chose these probability amplitudes because, when we square them, we still get the correct probabilities of measuring $0$ and $1$:

$$
\begin{aligned}
\text{For state } |+\rangle : \mathbb{P}_0 = \left(\frac{1}{\sqrt{2}}\right)^2 = \frac{1}{2} \quad &\text{and} \quad 
\mathbb{P}_1 = \left(\frac{1}{\sqrt{2}}\right)^2 = \frac{1}{2}.
\\
\\
\text{For state } |-\rangle : \mathbb{P}_0 = \left(\frac{1}{\sqrt{2}}\right)^2 = \frac{1}{2} \quad &\text{and} \quad  
\mathbb{P}_1 = \left(-\frac{1}{\sqrt{2}}\right)^2 = \frac{1}{2}.
\end{aligned}
$$

And secondly, the signs we selected for the probability amplitudes allowed us to express states $|0\rangle$ and $|1\rangle$ in terms of states $|+\rangle$ and $|-\rangle$ in a way that is consistent with observations when measurements of spin are performed along the $x$ axis. So, for state $|0\rangle$:

$$
\begin{aligned}
|0\rangle &= \frac{1}{\sqrt{2}}|+\rangle + \frac{1}{\sqrt{2}}|-\rangle 
\\
\\
&= \frac{1}{\sqrt{2}}\left(\frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}|1\rangle \right) + \frac{1}{\sqrt{2}}\left(\frac{1}{\sqrt{2}}|0\rangle - \frac{1}{\sqrt{2}}|1\rangle \right) 
\\
\\
&= \frac{1}{2}|0\rangle + \frac{1}{2}|1\rangle + \frac{1}{2}|0\rangle - \frac{1}{2}|1\rangle
\\
\\
&= |0\rangle
\end{aligned}
$$

Similarly, for state $|1\rangle$ we have:

$$
\begin{aligned}
|1\rangle &= \frac{1}{\sqrt{2}}|+\rangle - \frac{1}{\sqrt{2}}|-\rangle 
\\
\\
&= \frac{1}{\sqrt{2}}\left(\frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}|1\rangle \right) - \frac{1}{\sqrt{2}}\left(\frac{1}{\sqrt{2}}|0\rangle - \frac{1}{\sqrt{2}}|1\rangle \right) 
\\
\\
&= \frac{1}{2}|0\rangle + \frac{1}{2}|1\rangle - \frac{1}{2}|0\rangle + \frac{1}{2}|1\rangle
\\
\\
&= |1\rangle
\end{aligned}
$$

So, what do we do for the probability amplitudes of the states corresponding to spins along the $\pm y$ directions? Well, we need to follow similar rules to those we had for spin along $\pm x$ axes:

1. When "squared", they must be equal to probability values of $1/2$. 
2. We must be able to consistently express states $|0\rangle$ and $|1\rangle$ as superpositions of these states.

Unfortunately, there are no other **real** numbers that can meet these conditions. However, with just a few adjustments to the restrictions above, **complex** numbers can save the day! Therefore, the most general state that represents a qubit is of the form:

$$ 
\begin{aligned}
|q\rangle &= \begin{bmatrix} q_0 \\ q_1 \end{bmatrix}
\\
\\
|q\rangle &= q_0|0\rangle + q_1 |1\rangle, 
\end{aligned}
$$

with $q_i \in \mathbb{C}$, but where we need to make some changes to the conditions above if we want things to work.

To see what adjustments are needed, let's start by recalling that a complex number $c$ is of the form:

$$ c = a + bi, $$

where $a, b \in \mathbb{R}$, and $i$ is known as the imaginary unit, which satisfies the property that when squared, it equals negative one: $i^2 = -1 .$

We call $a$ and $b$ the real and imaginary parts of $c$, respectively. Complex numbers can then be graphically represented in what is known as the complex plane, with the horizontal direction representing the real axis, and the vertical direction the imaginary axis: 

<img src="images/02_03_03_complex_plane.png" align = "center" width="250"/>

With this in mind, we can make the very simple observation that, in this plane, the imaginary numbers $i$ and $-i$ are in an orthogonal number line to that of the real numbers, which includes the values $1$ and $-1$, and all of which lie in a circle of radius $1$:

<img src="images/02_03_04_i_in_plane.png" align = "center" width="250"/>

<a name="back1"></a> So, if we superimpose the complex plane on the $xy$-plane that defines the direction of the electron spin, we can see that we can associate a complex value in the unit circle with the direction of the spin statevector in this plane[$^*$](#footnote1):

<img src="images/02_03_05_spin_in_complex_plane.png" align = "center" width="250"/>

Furthermore, if we multiply this complex value with one of the probability amplitudes of our statevector, we will indeed get the full and accurate representation of a quantum bit. So, for example, states $|+\rangle$ and $|-\rangle$, which are aligned with the $\texttt{+}x$ and $\texttt{-}x$ directions, have one of their probability amplitudes multiplied by $+1$ and $-1$, respectively. Similarly, we can define the states for spin aligned with the $\texttt{+}y$ and $\texttt{-}y$ directions by multiplying one of their probability amplitudes by $+i$ and $-i$, accordingly:

$$ 
\begin{aligned}
|r\rangle &= \frac{1}{\sqrt{2}}|0\rangle + \frac{i}{\sqrt{2}}|1\rangle,
\\
\\
|l\rangle &= \frac{1}{\sqrt{2}}|0\rangle - \frac{i}{\sqrt{2}}|1\rangle.
\end{aligned}
$$

We label this states as $|r\rangle$ and $|l\rangle$ because, when looking at the three-dimensional Cartesian plane, they point in the "right" and "left" directions, respectively. This nomenclature is rather arbitrary, but at this point, it has become relatively standard.

Now, upon close inspection, these two statevectors do not really meet our probability condition! If we were two square the probability amplitude associated with $|1\rangle$, for let's say state $|r\rangle$, we would get an invalid probability because squaring $i$ gives us a negative value:

$$ \mathbb{P}_1 = \left(\frac{i}{\sqrt{2}}\right)^2 = -\frac{1}{2} \, \color{#E53F73}{\leftarrow \text{invalid probability}} .$$

Luckily, this has a simple fix. If instead of taking the square of probability amplitudes to find probability, we find its squared modulus, we will then always get a positive value. So, for an arbitrary complex number $c$, we find it's squared modulus by multiplying the number with its [complex conjugate](https://en.wikipedia.org/wiki/Complex_conjugate):

$$ 
\begin{aligned}
|c|^2 &= c\bar{c} = (a+bi)(a-bi) 
\\
\\
|c|^2 &= a^2 -abi + abi -b^2i^2
\\
\\
|c|^2 &= a^2 + b^2 
\end{aligned}
$$

Therefore, our updated (and final) version of the Born rule says that, to find the probability of measuring some given state we must take the squared modulus of the probability amplitude associated with it. Thus, for states $|r\rangle$ and $|l\rangle$ the probabilities of measuring $0$ and $1$ given below are consistent with what we observe from experimentation:

$$
\begin{aligned}
\text{For state } |r\rangle : \mathbb{P}_0 = \left|\frac{1}{\sqrt{2}}\right|^2 = \frac{1}{2} \quad &\text{and} \quad 
\mathbb{P}_1 = \left|\frac{i}{\sqrt{2}}\right|^2 = \frac{1}{2}.
\\
\\
\text{For state } |l\rangle : \mathbb{P}_0 = \left|\frac{1}{\sqrt{2}}\right|^2 = \frac{1}{2} \quad &\text{and} \quad  
\mathbb{P}_1 = \left|-\frac{i}{\sqrt{2}}\right|^2 = \frac{1}{2}.
\end{aligned}
$$

Now, let's inspect if we can recover states $|0\rangle$ and $|1\rangle$ by taking equal superpositions of these vectors, which is a condition derived from observations of running, for example, the Stern-Gerlach experiment with the magnetic field along the $y$ axis. So, first let's take a positive superposition:

$$
\begin{aligned}
\frac{1}{\sqrt{2}}|r\rangle + \frac{1}{\sqrt{2}}|l\rangle &= \frac{1}{\sqrt{2}}\left(\frac{1}{\sqrt{2}}|0\rangle + \frac{i}{\sqrt{2}}|1\rangle \right) + \frac{1}{\sqrt{2}}\left(\frac{1}{\sqrt{2}}|0\rangle - \frac{i}{\sqrt{2}}|1\rangle \right) 
\\
\\
&= \frac{1}{2}|0\rangle + \frac{i}{2}|1\rangle + \frac{1}{2}|0\rangle - \frac{i}{2}|1\rangle
\\
\\
&= |0\rangle,
\end{aligned}
$$

which clearly shows we can express $|0\rangle$ in terms of $|r\rangle$ and $|l\rangle$. And now if we take a negative superposition we get:

$$
\begin{aligned}
\frac{1}{\sqrt{2}}|r\rangle - \frac{i}{\sqrt{2}}|l\rangle &= \frac{1}{\sqrt{2}}\left(\frac{1}{\sqrt{2}}|0\rangle + \frac{i}{\sqrt{2}}|1\rangle \right) - \frac{1}{\sqrt{2}}\left(\frac{1}{\sqrt{2}}|0\rangle - \frac{i}{\sqrt{2}}|1\rangle \right) 
\\
\\
&= \frac{1}{2}|0\rangle + \frac{i}{2}|1\rangle - \frac{1}{2}|0\rangle + \frac{i}{2}|1\rangle
\\
\\
&= i|1\rangle.
\end{aligned}
$$

So we do not quite get $|1\rangle$ since it is premultiplied by a factor of $i$, but the thing is that states $|1\rangle$ and $i|1\rangle$ are effectively the equivalent. The reason being that, what we really want out of this quantum mechanical model is to be able to get accurate predictions of what we observe through experimentation. So we have to ask ourselves, what is expression $i|1\rangle$ telling me about what I will *measure* from this state? Well, we know that we will observe state $|1\rangle$ with a probability of $\mathbb{P}_1 = |i|^2 = 1$, which is the same prediction we will get from simply having state $|1\rangle$. We say these two states are "*equal up to a global phase*"; a statement that will become clear in the following sections.

So, this approach works for states $|r\rangle$ and $|l\rangle$, but what about the rest of possible spin directions in the $xy$-plane? So, to generalize this approach, all we need to do is assign to each state the corresponding number from the complex unit circle that lies along the same direction as the spin is pointing to. 

$$ (a + bi) = r e^{i \theta} $$

In [7]:
import numpy as np
from qiskit.quantum_info import Statevector

sv = np.sqrt(1/2)*Statevector.from_label('r') - np.sqrt(1/2)*Statevector.from_label('l')
sv

<IPython.core.display.Latex object>

## Footnotes

<a name="footnote1"></a><span style="font-size: smaller;">$^*$Note that here we are just abstractly superimposing the complex plane with the physical $xy$-plane to assign a complex number to the corresponding statevector associated with the spin in some given direction on this plane. We are **not** saying that this complex number is related to the physical length of the spin vector, or directly multiplying the full statevector. The relation between this complex value and the state is clarified in the section discussing the Bloch sphere. ([go back](#back1))